In [ ]:
import cv2
import cv2.aruco as aruco
import numpy as np
import math
import rospy
from geometry_msgs.msg import Pose2D
from scipy.spatial.transform import Rotation as R

# ===================== USER PARAMETERS =====================
CAM_DEV    = 2               # OpenCV device index
FRAME_W    = 1280            # camera resolution
FRAME_H    = 720
EXPOSURE   = 20              # exposure for low‑light ceilings (‑1 for auto)

MARKER_LEN = 0.16           # physical side length of every marker  [m]
TARGET_DICT = aruco.DICT_4X4_50

# ▶▶▶  사용자가 제공한 천장 마커 월드 좌표 (단위 m)
MARKER_WORLD = {
    0:  (0.0, 0.0, 0.0),  1:  (1.5, 0.0, 0.0),
    2:  (3.0, 0.0, 0.0),  3:  (4.5, 0.0, 0.0),
    4:  (0.0, 1.5, 0.0),  5:  (1.5, 1.5, 0.0),
    6:  (3.0, 1.5, 0.0),  7:  (4.5, 1.5, 0.0),
    8:  (0.0, 3.0, 0.0),  9:  (1.5, 3.0, 0.0),
    10: (3.0, 3.0, 0.0), 11: (4.5, 3.0, 0.0),
    12: (0.7, 0.7, 0.0), 13: (2.3, 0.7, 0.0),
    14: (3.7, 0.7, 0.0), 15: (0.7, 2.3, 0.0),
    16: (2.3, 2.3, 0.0), 17: (3.7, 2.3, 0.0),
}

# ▶▶▶  각 마커가 종이에 인쇄된 방향(시계 +, 단위 deg)
MARKER_YAW_DEG = {
    0:   0,   1:   0,   2: -180, 3:   0,
    4: -180, 5: -180, 6:   0,  7: -180,
    8:   0,   9:   0,  10: -180, 11:   0,
    12: -180, 13:  -90, 14: -270, 15:   0,
    16:  -90, 17:  -90,
}

# Robot (camera) height w.r.t world frame – DO NOT TOUCH unless ceiling height changes
FIXED_Z = -1.9               # [m] ← as requested, same as coord_v10.py
# ===========================================================


def get_camera_intrinsic():
    """Load *cameraMatrix* & *distCoeffs* from npz OR use fx,fy,cx,cy placeholders."""
    try:
        npz = np.load(rospy.get_param('~camera_calib_file'))
        return npz['cameraMatrix'], npz['distCoeffs']
    except Exception:
        # --- fallback: pinhole guess ---
        fx = fy = 800.0
        cx, cy = FRAME_W / 2.0, FRAME_H / 2.0
        K  = np.array([[fx,  0, cx],
                       [ 0, fy, cy],
                       [ 0,  0,  1]])
        D  = np.zeros(5)
        rospy.logwarn("[coord_center_pose] Using dummy intrinsics!")
        return K, D


def z_rot(deg: float) -> np.ndarray:
    """Return 3x3 rotation about +Z by *deg* degrees."""
    return R.from_euler('z', deg, degrees=True).as_matrix()


def main():
    rospy.init_node('coord_center_pose')
    pose_pub = rospy.Publisher('/robot_pose', Pose2D, queue_size=10)

    K, D = get_camera_intrinsic()

    cap = cv2.VideoCapture(CAM_DEV)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH,  FRAME_W)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_H)
    if EXPOSURE >= 0:
        cap.set(cv2.CAP_PROP_EXPOSURE, EXPOSURE)

    ar_dict  = aruco.getPredefinedDictionary(TARGET_DICT)
    ar_param = aruco.DetectorParameters()

    rate = rospy.Rate(30)

    while not rospy.is_shutdown():
        ret, frame = cap.read()
        if not ret:
            rate.sleep()
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners, ids, _ = aruco.detectMarkers(gray, ar_dict, parameters=ar_param)

        if ids is None:
            cv2.imshow('aruco', frame)
            cv2.waitKey(1)
            rate.sleep()
            continue

        rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(
            corners, MARKER_LEN, K, D
        )

        world_positions = []
        yaws            = []
        weights         = []

        for rvec, tvec, m_id in zip(rvecs, tvecs, ids.flatten()):
            m_id = int(m_id)
            if m_id not in MARKER_WORLD:
                continue

            # camera → marker
            R_cm, _ = cv2.Rodrigues(rvec)
            p_cm    = tvec.reshape(3)            # marker in camera frame

            # marker → world (static)
            p_wm    = np.array(MARKER_WORLD[m_id])
            R_wm    = z_rot(MARKER_YAW_DEG.get(m_id, 0.0))

            # world → camera
            R_wc    = R_wm @ R_cm.T
            p_wc    = p_wm - R_wc @ p_cm

            # 📌  FIXED Z — overrides whatever arithmetic gave us
            p_wc[2] = FIXED_Z

            # camera (==robot) in world
            world_positions.append(p_wc)
            yaws.append(R.from_matrix(R_wc).as_euler('xyz', degrees=False)[2])
            weights.append(1.0 / (np.linalg.norm(p_cm) + 1e-6))  # nearer marker = higher weight

            # draw axis for sanity
            aruco.drawAxis(frame, K, D, rvec, tvec, MARKER_LEN * 0.6)

        if not world_positions:
            cv2.imshow('aruco', frame)
            cv2.waitKey(1)
            rate.sleep()
            continue

        W = np.array(weights)
        W /= W.sum()

        pose_x = float(np.sum([w * p[0] for w, p in zip(W, world_positions)]))
        pose_y = float(np.sum([w * p[1] for w, p in zip(W, world_positions)]))
        pose_yaw = float(np.arctan2(np.sum(np.sin(yaws)*W),
                                    np.sum(np.cos(yaws)*W)))

        # Because camera centre == robot centre, we publish directly
        msg = Pose2D()
        msg.x = pose_x
        msg.y = pose_y
        msg.theta = pose_yaw
        pose_pub.publish(msg)

        # HUD
        txt = f"X={pose_x:.2f}  Y={pose_y:.2f}  Yaw={math.degrees(pose_yaw):.1f}°"
        cv2.putText(frame, txt, (30, 70), cv2.FONT_HERSHEY_SIMPLEX,
                    1.8, (0, 0, 255), 3)
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)
        cv2.imshow('aruco', frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

        rate.sleep()

    cap.release()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    try:
        main()
    except rospy.ROSInterruptException:
        pass
